In [1]:
# imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [6]:
# load training data
df = pd.read_csv('data/training_set_VU_DM.csv')

In [ ]:
df

In [ ]:
#df

pd.set_option('display.float_format', lambda x: '%.2f' % x)
# df.describe().loc[['min', 'max'], :].transpose()
df.describe().transpose()

In [ ]:
df.isnull().sum()
#df.columns[df.isna().any()].tolist()

In [ ]:
# number of unique values over all columns
df.nunique()

Features engineering and removal

In [12]:
# imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# load training data
df = pd.read_csv('data/training_set_VU_DM.csv')

In [3]:
from functions import comp_aggregation, remove_variables, impute_linear, impute_knn

df = comp_aggregation(df)
df = remove_variables(df)
df = impute_linear(df)

In [ ]:

df.isnull().sum()

In [30]:
from scipy.stats import boxcox
columns_to_check = [
    'prop_location_score1',
    'prop_location_score2',
    'prop_log_historical_price',
    'price_usd',
    'srch_length_of_stay',
    'srch_booking_window',
    'srch_adults_count',
    'srch_children_count',
    'srch_room_count',
    'orig_destination_distance',
    'comp_rate_ratio',
    'comp_inv_ratio'
]

for column in columns_to_check:
    # make negative values positive
    df[column] = df[column] - df[column].min() + 1
    transformed = pd.Series(boxcox(df[column])[0])
    # use iqr to remove outliers
    Q1 = transformed.quantile(0.25)
    Q3 = transformed.quantile(0.75)
    IQR = Q3 - Q1
    # use 3 for extreme outliers
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR

    # get index of outliers in transformed
    outliers = transformed[(transformed < lower_bound) | (transformed > upper_bound)].index

    print(f'{column} has {len(outliers)} outliers')
  


prop_location_score1 has 0 outliers
prop_location_score2 has 0 outliers
prop_log_historical_price has 0 outliers
price_usd has 7100 outliers
srch_length_of_stay has 0 outliers
srch_booking_window has 0 outliers
srch_adults_count has 1712113 outliers
srch_children_count has 1146573 outliers
srch_room_count has 438135 outliers
orig_destination_distance has 57 outliers
comp_rate_ratio has 1214187 outliers
comp_inv_ratio has 1214187 outliers


In [10]:
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

df.drop(columns="date_time", inplace=True)
df.drop(columns="gross_bookings_usd", inplace=True)

# predict position?
X = df.drop('position', axis=1)
y = df['position']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the training data to SVMLight format
dump_svmlight_file(X_train, y_train, 'train.txt')

# Convert the testing data to SVMLight format
dump_svmlight_file(X_test, y_test, 'test.txt')

dtrain = xgb.DMatrix('train.txt')

# Set LambdaMART parameters
params = {
    'objective': 'rank:ndcg',
    'eval_metric': 'ndcg@1-5',
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 1,
    'alpha': 0,
    'silent': 1,
    'nthread': 4
}

# Train the LambdaMART model using XGBoost
model = xgb.train(params, dtrain, num_boost_round=100)

# Load the testing data from the SVMLight file
dtest = xgb.DMatrix('test.txt')

# Make predictions using the trained LambdaMART model
y_pred = model.predict(dtest)

# Calculate NDCG score
ndcg = ndcg_score([y_test], [y_pred])

print("NDCG score:", ndcg)

# add predicted values to df
df['predicted_position'] = y_pred


Index(['srch_id', 'site_id', 'visitor_location_country_id', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool',
       'click_bool', 'gross_bookings_usd', 'booking_bool', 'comp_rate_ratio',
       'comp_inv_ratio'],
      dtype='object')
[16:31:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "silent" } are not used.

NDCG score: 0.9538586752551853
